In [ ]:
!pip install dart-fss

In [ ]:
import dart_fss as dart_fss
import pandas as pd

api_key = ''
dart_fss.set_api_key(api_key=api_key)

In [ ]:
# 전체종목

all=dart_fss.api.filings.get_corp_code()
df = pd.DataFrame(all)

In [ ]:
all[0]

In [ ]:
# 상장사 리스트

df_listed = df[df['stock_code'].notnull()]
df_listed.count()

In [ ]:
# 비상장사 리스트

df_non_listed = df[df['stock_code'].isnull()]
df_non_listed.count()

In [ ]:
# df_listed.to_excel('상장종목.xlsx')
# df_non_listed.to_excel('비상장종목.xlsx')

In [ ]:
# 기업 정보
corp_code = df_listed[df_listed['corp_name'] == '카카오'].iloc[0,0]
dart_fss.api.filings.get_corp_info(corp_code)
corp_code

In [ ]:
# 사업보고서 주요 정보

data = dart_fss.api.info.unrst_exctv_mendng_sttus(corp_code, '2021', '11011')
pd.DataFrame(data['list'])

In [ ]:
# 증자 현황

data = dart_fss.api.info.irds_sttus(corp_code, '2021', '11011')
pd.DataFrame(data['list'])

In [ ]:
# 배당 현황

data = dart_fss.api.info.alot_matter(corp_code, '2022', '11011')
pd.DataFrame(data['list'])

In [ ]:
# 최대 주주 현황

data = dart_fss.api.info.hyslr_sttus(corp_code, '2021', '11011')
pd.DataFrame(data['list'])

In [ ]:
# 임원 현황

corp_code = df_listed[df_listed['corp_name'] == '카카오'].iloc[0,0]
data = dart_fss.api.info.exctv_sttus(corp_code, '2021', '11011')

pd.DataFrame(data['list'])

In [ ]:
# 기초 소유 주식 지분율, 기말 소유 주식 지분율 현황

data = dart_fss.api.info.hyslr_sttus(corp_code, '2021', '11011')
pd.DataFrame(data['list'])[['nm','bsis_posesn_stock_qota_rt','trmend_posesn_stock_qota_rt']]

In [ ]:
# 직원, 남여 평균 급여 현황

corp_code = df_listed[df_listed['corp_name'] == '카카오'].iloc[0,0]
data = dart_fss.api.info.emp_sttus(corp_code, '2021', '11011')

pd.DataFrame(data['list'])

In [ ]:
# 이사 보수 현황

corp_code = df_listed[df_listed['corp_name'] == '카카오'].iloc[0,0]
data = dart_fss.api.info.hmv_audit_indvdl_by_sttus(corp_code, '2021', '11011')

pd.DataFrame(data['list'])

In [ ]:
# 연봉 Top5 현황

corp_code = df_listed[df_listed['corp_name'] == '카카오'].iloc[0,0]
data = dart_fss.api.info.indvdl_by_pay(corp_code, '2021', '11011')

pd.DataFrame(data['list'])

In [ ]:
# 재무제표 3년 주요 정보

corp_code = df_listed[df_listed['corp_name'] == '카카오'].iloc[0,0]
data = dart_fss.api.finance.fnltt_singl_acnt(corp_code, '2021', '11011')

pd.DataFrame(data['list'])


In [ ]:
# 재무제표 모든 주요 정보

corp_code = df_listed[df_listed['corp_name'] == '카카오'].iloc[0,0]
data = dart_fss.api.finance.fnltt_singl_acnt_all(corp_code, '2021', '11011', 'CFS')

pd.DataFrame(data['list'])

In [ ]:
# 임원, 주요 주주 소유 보고 현황

corp_code = df_listed[df_listed['corp_name'] == '카카오'].iloc[0,0]
data = dart_fss.api.shareholder.elestock(corp_code)

pd.DataFrame(data['list'])

In [ ]:
# 임원, 주주 중 특정인 소유 보고 현황

corp_code = df_listed[df_listed['corp_name'] == '카카오'].iloc[0,0]
data = dart_fss.api.shareholder.elestock(corp_code)

df_temp = pd.DataFrame(data['list'])
df_temp[df_temp['repror'] == '김범수']

In [ ]:
# 특정 회사의 연봉 추출

def get_salary(name):
  corp_code = df_listed[df_listed['corp_name'] == name].iloc[0,0]
  data = dart_fss.api.info.indvdl_by_pay(corp_code, '2021', '11011')

  df = pd.DataFrame(data['list'])
  df = df[['corp_name','nm','ofcps','mendng_totamt']]
  df.columns = ['기업명','이름','역할','보수']
  df['보수'] = pd.to_numeric(df['보수'].str.replace(',',''))
  # 백만원 단위에 콤마 추가
  df['보수'] = df['보수'].apply(lambda x: '{:,.0f}'.format(x))
  df = df.sort_values(by='보수',ascending=False)

  return df

In [ ]:
get_salary('삼성전자')

In [ ]:
get_salary('카카오')

In [ ]:
# 여러 종목의 연봉 추출

pd.options.display.float_format

names = ['삼성전자','LG에너지솔루션','SK하이닉스','NAVER','삼성바이오로직스','삼성전자우','카카오','삼성SDI','현대차','LG화학','기아','POSCO홀딩스','KB금융','카카오뱅크','셀트리온','신한지주','삼성물산','현대모비스','SK이노베이션','LG전자','카카오페이','SK','한국전력','크래프톤','하나금융지주','LG생활건강','HMM','삼성생명','하이브','두산중공업','SK텔레콤','삼성전기','SK바이오사이언스','LG','S-Oil','고려아연','KT&G','우리금융지주','대한항공','삼성에스디에스','현대중공업','엔씨소프트','삼성화재','아모레퍼시픽','KT','포스코케미칼','넷마블','SK아이이테크놀로지','LG이노텍','기업은행']

dfs = []

for name in names:
  try:
    df = get_salary(name)
    dfs.append(df)
  except:
    print(f'없음 - {name}')

df_result = pd.concat(dfs)
df_result.sort_values(by='보수',ascending=False).head(30)

In [ ]:
# 특정 회사의 최대주주 지분율

data = dart_fss.api.info.hyslr_sttus('00258801', '2021', '11011')
df = pd.DataFrame(data['list'])

df = df[['corp_name','nm','relate','bsis_posesn_stock_qota_rt','trmend_posesn_stock_qota_rt','rm']]
df.columns = ['회사명','이름','관계','기초지분율','기말지분율','비고']

df = df[df['관계'].notnull()]

df['기초지분율'] = pd.to_numeric(df['기초지분율'])
df['기말지분율'] = pd.to_numeric(df['기말지분율'])

df.sort_values(by='기초지분율',ascending=False).head(3)

In [ ]:
# 최대주주 지분율 함수화

def get_shareholders(corp_code):
  data = dart_fss.api.info.hyslr_sttus(corp_code, '2021', '11011')
  df = pd.DataFrame(data['list'])

  df = df[['corp_name','nm','relate','bsis_posesn_stock_qota_rt','trmend_posesn_stock_qota_rt','rm']]
  df.columns = ['회사명','이름','관계','기초지분율','기말지분율','비고']

  df = df[df['관계'].notnull()]

  df['기초지분율'] = pd.to_numeric(df['기초지분율'])
  df['기말지분율'] = pd.to_numeric(df['기말지분율'])

  return df.sort_values(by='기초지분율',ascending=False).head(3)

In [ ]:
get_shareholders('00258801')

In [ ]:
# 랜덤 10개 추출

df_listed.sample(10)

In [ ]:
# 랜덤 10개 추출의 최대주주 지분율

corp_codes = list(df_listed.sample(10)['corp_code'])

dfs = []
for corp_code in corp_codes:
  try:
    df = get_shareholders(corp_code)
    dfs.append(df)
  except:
    print(f'error - {corp_code}')

df_result = pd.concat(dfs)

df_result['증감'] = df_result['기말지분율'] - df_result['기초지분율']

df_result.sort_values(by='증감',ascending=False)

In [ ]:
# 이이익잉여금 변화 함수화

def get_profit(corp_name):
  corp_code = df_listed[df_listed['corp_name'] == corp_name].iloc[0,0]
  data = dart_fss.api.finance.fnltt_singl_acnt(corp_code, '2021', '11011')
  df = pd.DataFrame(data['list'])

  df = df[df['fs_div'] == 'CFS']

  df['name'] = corp_name
  cond = df['account_nm'] == '이익잉여금'
  df = df[cond]

  df = df[['name','thstrm_amount','frmtrm_amount']]
  df.columns = ['이름','당기','전기']

  df['당기'] = pd.to_numeric(df['당기'].str.replace(',',''))
  df['전기'] = pd.to_numeric(df['전기'].str.replace(',',''))

  df['증감'] = df['당기'] - df['전기']
  df['증감율'] = abs(df['증감'])/abs(df['전기'])
  
  # 백만원 단위에 콤마 추가
  df['당기'] = df['당기'].apply(lambda x: '{:,.0f}'.format(x))
  df['전기'] = df['전기'].apply(lambda x: '{:,.0f}'.format(x))
  df['증감'] = df['증감'].apply(lambda x: '{:,.0f}'.format(x))

  return df

In [ ]:
get_profit('현대자동차')

In [ ]:
# 랜덤 10개 추출 기업의 이익잉여금 변화

names = list(df_listed.sample(10)['corp_name'])

dfs = []
for name in names:
  try:
    df = get_profit(name)
    dfs.append(df)
  except:
    print(f'error - {name}')

df_result = pd.concat(dfs)
df_result

In [ ]:
# 랜덤 100개 추출 기업의 이익잉여금 변화

names = list(df_listed.sample(100)['corp_name'])

dfs = []
for name in names:
  try:
    df = get_profit(name)
    dfs.append(df)
  except:
    print(f'error - {name}')

df_result = pd.concat(dfs)
df_result.sort_values(by='증감율',ascending=False)

In [ ]:
# 연결 포괄손익계산서 지배주주당기순이익 정보
# 연결재무제표는 종속회사와 모회사를 하나로 보고 작성하는 재무제표
# 종속회사의 기준 = 자회사의 지분율이 50%를 넘거나 자회사 지분이 30% 를 넘으면서 중요한 영향력을 행사할 수 있을 경우

corp_code = df_non_listed[df_non_listed['corp_name'] == '야놀자'].iloc[0,0]
data = dart_fss.api.info.alot_matter(corp_code, '2021', '11011')
df = pd.DataFrame(data['list'])

df = df[df['se'] == '(연결)당기순이익(백만원)'][['corp_name','thstrm','frmtrm','lwfr']]
df.columns = ['기업명','2021','2020','2019']

df['2019'] = pd.to_numeric(df['2019'].str.replace(',',''))
df['2020'] = pd.to_numeric(df['2020'].str.replace(',',''))
df['2021'] = pd.to_numeric(df['2021'].str.replace(',',''))

# 백만원 단위에 콤마 추가
df['2019'] = df['2019'].apply(lambda x: '{:,.0f}'.format(x))
df['2020'] = df['2020'].apply(lambda x: '{:,.0f}'.format(x))
df['2021'] = df['2021'].apply(lambda x: '{:,.0f}'.format(x))
  
df

In [ ]:
# 배당 정보 함수화

def get_earning(name):
  corp_code = df_non_listed[df_non_listed['corp_name'] == name].iloc[0,0]
  data = dart_fss.api.info.alot_matter(corp_code, '2021', '11011')
  df = pd.DataFrame(data['list'])

  df = df[df['se'] == '(연결)당기순이익(백만원)'][['corp_name','thstrm','frmtrm','lwfr']]
  df.columns = ['기업명','2023','2022','2021']

  df['2021'] = pd.to_numeric(df['2021'].str.replace(',',''))
  df['2022'] = pd.to_numeric(df['2022'].str.replace(',',''))
  df['2023'] = pd.to_numeric(df['2023'].str.replace(',',''))
  
  # 백만원 단위에 콤마 추가
  df['2021'] = df['2021'].apply(lambda x: '{:,.0f}'.format(x))
  df['2022'] = df['2022'].apply(lambda x: '{:,.0f}'.format(x))
  df['2023'] = df['2023'].apply(lambda x: '{:,.0f}'.format(x))

  return df

In [ ]:
get_earning('비바리퍼블리카')

In [ ]:
# 직원 정보

corp_code = df_listed[df_listed['corp_name'] == '카카오'].iloc[0,0]
data = dart_fss.api.info.emp_sttus(corp_code, '2021', '11011')

pd.DataFrame(data['list'])

In [ ]:
# 남여 연봉 차이 함수화

def get_salary(corp_code):
  data = dart_fss.api.info.emp_sttus(corp_code, '2021', '11011')

  df = pd.DataFrame(data['list'])
  df = df[['corp_name','sexdstn','jan_salary_am']]

  df_result = pd.DataFrame()
  doc = {
      '기업명':df.iloc[0,0],
      '연봉(남)':df[df['sexdstn'] == '남'].iloc[0,2],
      '연봉(여)':df[df['sexdstn'] == '여'].iloc[0,2]
  }
  df_result = df_result.append(doc, ignore_index=True)

  df_result['연봉(남)'] = pd.to_numeric(df_result['연봉(남)'].str.replace(',',''))
  df_result['연봉(여)'] = pd.to_numeric(df_result['연봉(여)'].str.replace(',',''))

  return df_result

In [ ]:
get_salary('00258801')

In [ ]:
# 랜덤 10개 기업 추출 후 남여 연봉 차이

corp_codes = list(df_listed.sample(10)['corp_code'])

dfs = []
for corp_code in corp_codes:
  try:
    df = get_salary(corp_code)
    dfs.append(df)
  except:
    print(f'error - {corp_code}')

df_result = pd.concat(dfs)

df_result['차이(남-여)'] = df_result['연봉(남)'] - df_result['연봉(여)']
df_result['평균'] = (df_result['연봉(남)']+df_result['연봉(여)'])/2

df_result.sort_values(by="차이(남-여)",ascending=True)